In [42]:
%load_ext autoreload
%autoreload 2

import numpy as np
import torch
from torch import nn 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Teaching a quadruped to walk

Time to try out the learning algorithms that you just implemented on a more difficult problem. The WalkerEnv implements a quadruped robot kind-of thing, see for yourself. The goal is to move in the $x$ direction as fast and as far as possible.

Your goal is to implement a class `WalkerPolicy` with function `determine_actions()` just like the StochasticPolicy we used earlier to control the pendulum. Below is a template of this class, but feel free to alter it however you want. The only important thing is the `determine_actions()` function!

After you implement it, copy `WalkerPolicy` into a separate file `WalkerPolicy.py` that you will upload to BRUTE together with the (optional) learned weights in a zip file. How the policy is implemented is up to you! You are constrained to only the libraries we used so far though, such as torch, numpy etc..

You will get some free points just for uploading a working policy (irrelevant of the performance). Further 2 points will be awarded for successfully traversing a small distance in the x direction.


# Hints

There is no single easy way of doing this, but here are some suggestions on what you could try to improve your policy:

1. This problem is much more difficult, than balancing a pendulum. It is a good idea to use a bit larger network than for the pendulum policy.

2. You can also try to use a different optimizer, such as Adam and play with the hyperparameters.

3. Using a neural network to compute the normal distribution scale $\sigma$ can lead to too much randomness in the actions (i.e. exploration). You can use a fixed $\sigma$ instead, or replace it with a learnable `torch.Parameter` initialized to some small constant. Make sure, you run it through an exponential, or softplus function to ensure $\sigma$ is positive.

4. The exploration can also be reduced by penalizing the variance of the action distribution in an additional loss term.

5. If you see some undesirable behaviour, you can tweak the reward function to penalize it. Even though the $x$ distance is all we care about, adding extra terms to the reward can help guide the learning process (This is known as reward shaping). Simply define a reward function mapping the state $s_{t+1}$ and action $a_t$ to a scalar reward $r_t$ and put it in the config dictionary under the key `'reward_fcn'`. See the `WalkerEnv` class for the implementation of the default reward.

6. Using the normal distribution on a bounded action space can lead to certain problems caused by action clipping. This can be mitigated by using a different distribution, such as the Beta distribution. See the `torch.distributions.beta` module for more information. (Note that Beta distribution is defined on the interval [0,1] and works better with parameters $\alpha,\beta \geq 1$.)


In [48]:
# If you cannot run with the visualization, you can set this to False
VISUALIZE = True

In [49]:
from environment.WalkerEnv import WalkerEnv
from WalkerPolicy import WalkerPolicy

In [50]:
def walker_reward(state, action):
    """reward function for the walker environment, state is [29] vector, action is [8] vector"""
    pos = state[:15]  # first 15 elements of state vector are generalized coordinates [xyz, quat, joint_angles]
    vel = state[15:]  # last 14 elements of state vector are generalized velocities [xyz_vel, omega, joint_velocities]
    return vel[0]  # return the x velocity as the reward by default

In [51]:
# This is the configuration for the Walker environment
# N is the number of robots controlled in parallel
# vis is a boolean flag to enable visualization
# !! IMPORTANT track is an integer index to enable camera tracking of a particular robot (indexed by the value of the argument), this is useful when evaluating the performance of the policy after training
# reward_fcn is the reward function that the environment will use to calculate the reward
config = {'N': 1, 'vis': VISUALIZE, "track": 0, "reward_fcn": walker_reward}
env = WalkerEnv(config)

Environment ready


c:\Users\bogda\Deep Reinforcement Learning\hw5-rl-main\rl-homework-venv\lib\site-packages\glfw\__init__.py:917: GLFWError: (65537) b'The GLFW library is not initialized'
  warnings.warn(message, GLFWError)


In [53]:
# T = 1000
# obs = env.vector_reset()  # Observation vector is of shape (N, 29)
# print(f"Observation shape is {obs.shape}")
# x = obs[0, 0]
# for i in range(1000):
#     a = np.random.randn(1, 8) * 4 - 2  # Random actions with standard deviation of 2 and mean of 0
#     obs, reward = env.vector_step(a)
#     x = max(x, obs[0, 0])
# env.close()
# print(f"After {T} steps, the maximum x value reached was {x}")




# env = WalkerEnv(config)

# # Initialize policy and optimizer
# policy = WalkerPolicy(state_dim=29, action_dim=8)
# optimizer = optim.Adam(policy.parameters(), lr=3e-4)

# # Training hyperparameters
# num_episodes = 1000
# max_steps_per_episode = 500
# gamma = 0.99

# # Training loop
# for episode in range(num_episodes):
#     states, actions, rewards, log_probs = [], [], [], []
#     obs = env.vector_reset()  # Reset the environment

#     for t in range(max_steps_per_episode):
#         # Convert observation to tensor
#         obs_torch = torch.tensor(obs, dtype=torch.float32)

#         # Sample actions and log-probabilities
#         action, log_prob = policy.sample_actions_and_log_prob(obs_torch)

#         # Step the environment
#         obs_next, reward = env.vector_step(action.detach().numpy())

#         # Store experience
#         states.append(obs_torch)
#         actions.append(action)
#         rewards.append(torch.tensor(reward, dtype=torch.float32))
#         log_probs.append(log_prob)

#         obs = obs_next

#     # Compute discounted returns
#     rewards = torch.stack(rewards)  # Shape (T, 1)
#     discounted_returns = torch.zeros_like(rewards)
#     running_sum = 0.0

#     for t in reversed(range(len(rewards))):
#         running_sum = rewards[t] + gamma * running_sum
#         discounted_returns[t] = running_sum

#     # Prepare data for policy gradient computation
#     states = torch.stack(states)
#     actions = torch.stack(actions)
#     log_probs = torch.cat(log_probs)
#     returns = discounted_returns

#     # Compute loss
#     policy_loss = - (log_probs * returns).mean()

#     # Update policy
#     optimizer.zero_grad()
#     policy_loss.backward()
#     optimizer.step()

#     # Logging
#     total_return = returns.sum().item()
#     print(f"Episode {episode + 1}, Total Return: {total_return:.2f}")

# policy.save_weights("walker_weights.pt")

# env.close()
# print("Training complete.")

env = WalkerEnv(config)

# Initialize policy and load pretrained weights
policy = WalkerPolicy(state_dim=29, action_dim=8, load_weights=True)
policy.load_weights("walker_weights.pt")  # Load the pretrained weights

# Validation hyperparameters
num_validation_episodes = 10
max_steps_per_episode = 500

# Validation loop
for episode in range(num_validation_episodes):
    obs = env.vector_reset()  # Reset the environment
    total_return = 0.0

    for t in range(max_steps_per_episode):
        # Convert observation to tensor
        obs_torch = torch.tensor(obs, dtype=torch.float32)

        # Use the trained policy to determine actions
        action = policy.determine_actions(obs_torch)

        # Step the environment
        obs_next, reward = env.vector_step(action.detach().numpy())

        # Accumulate reward
        total_return += reward[0]
        obs = obs_next

    print(f"Validation Episode {episode + 1}, Total Return: {total_return:.2f}")

env.close()
print("Validation complete.")


Environment ready
Episode 1, Total Return: 2939.12
Episode 2, Total Return: -431.21
Episode 3, Total Return: -1619.18
Episode 4, Total Return: -2756.68
Episode 5, Total Return: -1479.39
Episode 6, Total Return: 2897.80
Episode 7, Total Return: 4312.67
Episode 8, Total Return: -106.67
Episode 9, Total Return: -1266.22
Episode 10, Total Return: -26.32
Episode 11, Total Return: 3142.88
Episode 12, Total Return: 1223.46
Episode 13, Total Return: 2351.47
Episode 14, Total Return: 3576.44
Episode 15, Total Return: 2133.28
Episode 16, Total Return: -85.05
Episode 17, Total Return: -951.43
Episode 18, Total Return: 2397.81
Episode 19, Total Return: -2732.25
Episode 20, Total Return: -1420.50
Episode 21, Total Return: 484.35
Episode 22, Total Return: -1708.03
Episode 23, Total Return: -2056.43
Episode 24, Total Return: 2566.36
Episode 25, Total Return: 3249.27
Episode 26, Total Return: 2135.59
Episode 27, Total Return: 603.66
Episode 28, Total Return: 1141.60
Episode 29, Total Return: -1593.38
